In [1]:
import pandas as pd
import urllib.request as ur
from datetime import date, timedelta, datetime
import os

In [2]:
### ### ### CHANGE WORKING DIRECTORY TO OUR PIPELINE STORAGE ### ### ### 
print('Old Working Directory:',os.getcwd())
os.chdir('/media/tom/Data2/Alpha Hound/Equity Algo/Exchange FTP')
print('New Working Directory:',os.getcwd())

Old Working Directory: /home/tom/Python
New Working Directory: /media/tom/Data2/Alpha Hound/Equity Algo/Exchange FTP


In [3]:
today = date.today()
#nasdaq_date = today.strftime('%Y%m%d')
#nyse_date = today.strftime('%d-%b-%Y')

In [4]:
##generator function below taken from stack overflow https://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
#generate a range of datetimes to iterate over for the data fetch
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [5]:
df = pd.read_csv('SSR_circuit_breakers.csv')
df = df.sort_values(by='date')
last_date = df['date'].iloc[-1]
last_date = datetime.strptime(last_date,'%Y-%m-%d')
#df['date'] = pd.to_datetime(df['date'])

/home/tom/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
start_date = last_date.date()
end_date = today + timedelta(days=1)

In [7]:
nasdaq_fails = []
nyse_fails = []

for single_date in daterange(start_date, end_date):
    try:
        query_date = single_date.strftime("%Y%m%d")
        url = 'ftp://ftp.nasdaqtrader.com/SymbolDirectory/shorthalts/shorthalts{0}.txt'.format(query_date)
        req = ur.Request(url)
        data = ur.urlopen(req)
        data = pd.read_csv(data,sep=",")
        data.drop(data.tail(1).index,inplace=True)
        data['Trigger Time'] = pd.to_datetime(data['Trigger Time'])
        data['date'] = single_date
        data['exchange'] = 'NASDAQ'
        #print(data)
        df = df.append(data,ignore_index=True)
    except:
        nasdaq_fails.append(single_date)
    try:
        year = single_date.strftime('%Y')
        year_month = single_date.strftime('%Y%m')
        query_date = single_date.strftime('%Y%m%d')
        url = 'ftp://ftp.nyse.com/NYSEGroupSSRCircuitBreakers/NYSEGroupSSRCircuitBreakers_{}/NYSEGroupSSRCircuitBreakers_{}/NYSEGroupSSRCircuitBreakers{}.xls'.format(year,year_month,query_date)
        data = pd.read_excel(url)
        data = data.rename(columns = {'Issue Symbol':'Symbol','Issue Name':'Security Name'})
        data['Trigger Time'] = data['Trigger Date'] + ' '+ data['Trigger Time']
        data['Trigger Time'] = pd.to_datetime(data['Trigger Time'])
        data = data.drop(columns='Trigger Date')
        #print(single_date)
        data['date'] = single_date
        data['exchange'] = 'NYSE'
        #print(data)
        df = df.append(data,ignore_index=True)
    except:
        nyse_fails.append(single_date)

In [8]:
df.to_csv('SSR_circuit_breakers.csv',index=False)

In [9]:
print(len(nasdaq_fails))
print(len(nyse_fails))

0
1
